a short post on using DomoLibrary to create and update custom roles with new grant lists and auto assign users to that role

# Project Configuration
## ⚙️ configure environment variables

This script assumes the use of a dotenv file (in this example `sample_config.txt`)

In [ ]:
# pip install python-dotenv

In [ ]:
from dotenv import dotenv_values

env = dotenv_values("sample_config.txt")
env

OrderedDict([('ROLE_NAME', 'dl_department_admin'),
             ('ROLE_DESCRIPTION', 'deployed via domo_library script'),
             ('ROLE_GRANTS',
              'alert.edit, alert.actions, content.card.embed, content.export, content.variable.edit, audit, datastore.create, dataset.manage, dataset.export, publish.subscribers.manage, user.invite, group.edit, certifiedcontent.admin, certifiedcontent.request'),
             ('ROLE_EMAILS', 'test1@test.com, test2@test.com'),
             ('ROLE_NAME2', 'dl_test'),
             ('ROLE_DESCRIPTION2', 'deployed via domo_library script'),
             ('ROLE_GRANTS2', 'alert.edit, alert.actions, content.card.embed'),
             ('ROLE_EMAILS2',
              'test3@test.com, test3@test.com, test4@test.com')])

## ⚙️ Creds config and roles to create

the domolibrary features a class based and function based approach to interacting with domo entities.

use the `domolibrary.client.DomoAuth` objects to handle api authentication 

access_tokens can be configured in Domo > Auth > Security > Access Token and have the benefit of not requiring direct signon access in environments that are using SSO

In [ ]:
# pip install domolibrary --upgrade

In [ ]:
import domolibrary.client.DomoAuth as dmda
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await token_auth.get_auth_token()

assert isinstance(token_auth.token, str)

## Templatize user input with classes

The custom `EnvRole` class allows users to define configuration in the `.env` file; however ensures conformity and reduces code redundancy by templatizing the required input.

In [ ]:
from pprint import pprint
from dataclasses import dataclass
import domolibrary.classes.DomoGrant as dmg


@dataclass
class EnvRole:
    name: str
    description: str
    grant_ls: [
        dmg.DomoGrant
    ]  # grants are consistent across domo instances so can be defined on initialization
    user_ls: [
        str
    ]  # each instance would have a diferent user_id associated with each instance so should be handled on a per instance basis (DomoUsers expect a set user id)

    """custom class for templatizing roles to create"""

    def __init__(
        self,
        name: str,
        description: str,
        grants_str: str,  # comma separated string of grant_ids
        user_str: [str],
    ):
        self.name = name
        self.description = description
        self.grant_ls = [
            dmg.DomoGrant(id=grant.strip()) for grant in grants_str.split(",")
        ]
        self.user_ls = [user.strip() for user in user_str.split(",")]

        # List of roles that will be created


roles_to_create = [
    EnvRole(
        name=env["ROLE_NAME"],
        description=env["ROLE_DESCRIPTION"],
        grants_str=env["ROLE_GRANTS"],
        user_str=env["ROLE_EMAILS"],
    ),
    EnvRole(
        name=env["ROLE_NAME2"],
        description=env["ROLE_DESCRIPTION2"],
        grants_str=env["ROLE_GRANTS2"],
        user_str=env["ROLE_EMAILS2"],
    ),
]

pprint(roles_to_create)

[EnvRole(name='dl_department_admin',
         description='deployed via domo_library script',
         grant_ls=[DomoGrant(id='alert.edit',
                             display_group=None,
                             title=None,
                             depends_on_ls=None,
                             description=None,
                             role_membership_ls=None),
                   DomoGrant(id='alert.actions',
                             display_group=None,
                             title=None,
                             depends_on_ls=None,
                             description=None,
                             role_membership_ls=None),
                   DomoGrant(id='content.card.embed',
                             display_group=None,
                             title=None,
                             depends_on_ls=None,
                             description=None,
                             role_membership_ls=None),
                   DomoGrant(id='c

## Define Functions that bridge the EnvRole with `domolibrary` classes

In the examples below, the functions are very simple and just call the API with passthrough parameters; however, more customization could be added for example defining a default role_description if one wasn't provided.

Notice how `upsert_super_admin` doesn't even accept a list of grants and instead pulls a list of all available grants from that Domo Instance.

This might be necessary because Domo by default doesn't grant all grants to the Admin role.

In [ ]:
import domolibrary.classes.DomoRole as dmr
import domolibrary.client.DomoAuth as dmda


async def upsert_role(
    auth: dmda.DomoAuth,
    role_name: str,
    role_description: str,
    grant_ls: [dmg.DomoGrant],
    debug_api: bool = False,
    debug_prn: bool = False,
):

    return await dmr.DomoRoles.upsert_role(
        auth=auth, name=role_name, description=role_description, grant_ls=grant_ls
    )

#### sample implementation of upsert_role

In [ ]:
import datetime as dt

role = roles_to_create[0]

await upsert_role(
    auth=token_auth,
    role_name=role.name,
    role_description=f"{role.description} - updated {dt.date.today()}",
    grant_ls=role.grant_ls,
    debug_prn=True,
)

[DomoGrant(id='alert.edit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='alert.actions', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.card.embed', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.export', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.variable.edit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='audit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='datastore.create', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='dataset.manage', display_group=None, title=None, depends_on_ls=None, description=None, role_membe

DomoRole(id=1563101750, name='dl_department_admin', description='deployed via domo_library script - updated 2023-03-23', is_system_role=0, is_default_role=False, grant_ls=[DomoGrant(id='alert.edit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='dataset.manage', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.card.embed', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='certifiedcontent.admin', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='datastore.create', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='group.edit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='publish.subscribers.manage', display_group=Non

In [ ]:
import datetime as dt
import domolibrary.classes.DomoInstanceConfig as dmic


# grants for super admin role are getting directly from instance using get_all_instance_grants
async def upsert_super_admin(
    auth: dmda.DomoAuth,
    role_name: str,
    role_description=f"all grants - updated on {dt.date.today()}",
    debug_api: bool = False,
    debug_prn: bool = False,
):

    domo_instance = dmic.DomoInstanceConfig(auth=auth)
    all_grants = await domo_instance.get_grants()

    sa_role = await dmr.DomoRoles.upsert_role(
        name=role_name,
        description=role_description,
        auth=auth,
        debug_api=debug_api,
        grant_ls=all_grants,
    )

    return sa_role

#### sample implementation of creating a super_admin role

In [ ]:
await upsert_super_admin(auth=token_auth, role_name="super_admin")

[DomoGrant(id='authorization.roles', display_group='COMPANY', title='Manage all roles', depends_on_ls=['ui.admin'], description='Create, edit and delete custom roles. Assign anyone to any role in this instance.', role_membership_ls=['1', '275763436', '810756122']), DomoGrant(id='audit', display_group='COMPANY', title='View activity logs', depends_on_ls=['ui.admin'], description='View and export audit logs.', role_membership_ls=['1', '275763436', '810756122', '1927158482', '1563101750', '2097317660']), DomoGrant(id='customer.edit', display_group='COMPANY', title='Manage all company settings', depends_on_ls=['ui.admin'], description='Manage company metadata, authentication rules, authorized domain rules, export security, mobile home screen layouts, publishing, SSO Configuration, whitelist rules, report scheduler and licenses.', role_membership_ls=['1', '275763436', '810756122']), DomoGrant(id='developer.token.manage', display_group='COMPANY', title='Manage all access tokens and API clien

DomoRole(id=810756122, name='super_admin', description='updated via domolibrary on 2023-03-24', is_system_role=0, is_default_role=False, grant_ls=[DomoGrant(id='versions.deployment.manage', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='buzz.admin', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='dataflow.admin', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='goal.admin', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='dataset.admin', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='workflow.admin', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='account.view.all', display_group=None, title=None, depends_on_ls=None, des

## create user

At code execution, it is possible that a user may need to be a specific role, but that user and the role haven't been deployed to the instance yet.

The proper order of operations would be to create a role and then assign the user to that role.  You cannot create a user without defining their role membership

In [ ]:
import domolibrary.classes.DomoUser as dmu


async def upsert_user(
    auth: dmda.DomoAuth,
    email_address: str,
    role_id: str,
    debug_api: bool = False,
):

    return await dmu.DomoUsers.upsert_user(
        email_address=email_address, role_id=role_id, auth=auth, debug_api=debug_api
    )

#### sample implementation of upsert_user

In [ ]:
role = roles_to_create[0]
await upsert_user(auth=token_auth, role_id=4, email_address="test23@test.com")

ResponseGetData(status=200, response={'attributes': [{'key': 'id', 'values': [1802840904]}, {'key': 'displayName', 'values': ['test23@test.com - via dl 2023-03-16']}, {'key': 'userName', 'values': ['test23@test.com']}, {'key': 'emailAddress', 'values': ['test23@test.com']}, {'key': 'modified', 'values': [1680526181615]}, {'key': 'created', 'values': [1679010842000]}, {'key': 'roleId', 'values': [4]}, {'key': 'isAnonymous', 'values': [True]}, {'key': 'isSystemUser', 'values': [False]}, {'key': 'isPending', 'values': [True]}, {'key': 'isActive', 'values': [True]}, {'key': 'invitorUserId', 'values': [1893952720]}, {'key': 'avatarKey', 'values': ['/api/content/v1/avatar/USER/1802840904']}], 'id': 1802840904, 'displayName': 'test23@test.com - via dl 2023-03-16', 'roleId': 4, 'userName': 'test23@test.com', 'emailAddress': 'test23@test.com'}, is_success=True)

# Main Implementation
With one function, `create_role` which calls upsert_role` and `upsert_user` we can quickly map over the `EnvRole` class and rapidly deploy a consistent set of roles to one or many domo instances just by swapping out the `DomoAuth` object.

Notice the intentional use of `asyncio` to execute code asynchronously where appropriate.  We must deploy a role before deploying users, but we could configure multiple roles simultaneously because they are not dependent.

In [ ]:
import datetime as dt
import asyncio


async def create_role(
    role_name, role_description, grant_ls, email_ls, auth, debug_prn: bool = False
):
    if debug_prn:
        pprint(
            {
                "name": role_name,
                "description": role_description,
                "grant_ls": grant_ls,
                "email_ls": email_ls,
            }
        )

    domo_role = await upsert_role(
        auth=auth,
        role_name=role_name,
        role_description=f"{role_description} - updated {dt.date.today()}",
        grant_ls=grant_ls,
    )

    return await asyncio.gather(
        *[
            upsert_user(auth=auth, role_id=domo_role.id, email_address=email_address)
            for email_address in email_ls
        ]
    )

#### sample implementation of create_role

In [ ]:
await asyncio.gather(
    *[
        create_role(
            role_name=role.name,
            role_description=role.description,
            grant_ls=role.grant_ls,
            email_ls=role.user_ls,
            auth=token_auth,
        )
        for role in roles_to_create
    ]
)

[DomoGrant(id='alert.edit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='alert.actions', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.card.embed', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.export', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='content.variable.edit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='audit', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='datastore.create', display_group=None, title=None, depends_on_ls=None, description=None, role_membership_ls=None), DomoGrant(id='dataset.manage', display_group=None, title=None, depends_on_ls=None, description=None, role_membe

[[ResponseGetData(status=200, response={'attributes': [{'key': 'id', 'values': [308783524]}, {'key': 'displayName', 'values': ['test1']}, {'key': 'userName', 'values': ['test1@test.com']}, {'key': 'emailAddress', 'values': ['test1@test.com']}, {'key': 'modified', 'values': [1680526185953]}, {'key': 'created', 'values': [1664931963000]}, {'key': 'roleId', 'values': [1563101750]}, {'key': 'isAnonymous', 'values': [True]}, {'key': 'isSystemUser', 'values': [False]}, {'key': 'isPending', 'values': [True]}, {'key': 'isActive', 'values': [True]}, {'key': 'invitorUserId', 'values': [1893952720]}, {'key': 'avatarKey', 'values': ['/api/content/v1/avatar/USER/308783524']}], 'id': 308783524, 'displayName': 'test1', 'roleId': 1563101750, 'userName': 'test1@test.com', 'emailAddress': 'test1@test.com'}, is_success=True),
  ResponseGetData(status=200, response={'attributes': [{'key': 'id', 'values': [894540685]}, {'key': 'displayName', 'values': ['test2@test.com - via dl 2023-03-16']}, {'key': 'userN